In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import pickle
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib import animation
from matplotlib.collections import LineCollection

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float32)
print("Running on:", device)

### Regularised Version

In this file we will note down the regularised training procedure. This will then be used in the scripts later. Note that the regularised training procedure is different in architecture, using mean-field neural networks (note the mean-field scaling). 

In [ ]:
# hyperparameters and wind generator function
n = 50
T = 50
L_x = 20
theta = 0.1
mu = 0
wind_sigma = 0.05
tau = torch.tensor(1, device = device)
# we will discretise wind on a finer grid, then choose only the relevant points
# assuming that the trajectory is discretised with time-step 1
def wind_process(T, theta, mu, wind_sigma, n, tau):
    num_to_sim = int(T / tau)
    winds = torch.zeros(n, num_to_sim + 1, device = device)
    winds[:, 0] = 0.5 * torch.rand(n , device = device) - 0.25 
    for step in range(1, num_to_sim + 1):
        dW = torch.randn(n , device = device)
        winds[:, step] = winds[:, step - 1] +(mu - winds[:, step - 1]) * theta * tau + wind_sigma * torch.sqrt(tau) * dW
    # only return winds for the integer time-steps
    final_wind = winds[:, 1:][:, ::int(1 / tau)]
    # add initial wind to the front
    winds = torch.cat((winds[:, 0].view(n, 1), final_wind), dim = 1)
    return winds

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_dim, width, output_dim):
        super(NeuralNet, self).__init__()
        self.hidden_layer = nn.Linear(input_dim, width)
        # self.hidden_layer.bias.data.zero_()
        # self.hidden_layer.bias.requires_grad = False
        self.sigmoid = nn.Tanh()
        self.output_layer = nn.Linear(width, output_dim)
        # self.output_layer.bias.data.zero_()
        # self.output_layer.bias.requires_grad = False
        self.width = width

    def forward(self, x):
        hidden_activations = self.sigmoid(self.hidden_layer(x))
        unscaled = self.output_layer(hidden_activations)
        return unscaled# / self.width